# Suffix Tree Term



#### Made by Yuliia Hetman

In [1]:
%%time
from multiprocessing import Pool
import re, os, json
from string import digits
import pandas as pd

docIDs = list(range(0, 18))

def delete_sings_digits(content):
    punc = '''!()-'”№[]{};:"\,“«»<>./?@#$%—…^&*_~|–abcdefghijklmnoqprstuvwxyz'''
    content = "".join([ele for ele in content.lower() if ele not in punc])
    table = str.maketrans('', '', digits)
    content = content.translate(table)
    return content


def create_vocabulary(content):    
    raw_content = delete_sings_digits(content)
    vocab = [i.lower() for i in raw_content.split() if i != '']
    return vocab

def read_book(book):
    dirname='../books/'
    with open(dirname + book, 'r', encoding='utf-8') as file:
        text = create_vocabulary(file.read())
    return text

        
def get_vocabulary(dirname='../books/'):
    books = os.listdir(dirname)
    vocabulary = list()
    dictionary = dict()
    memory = 0
    with Pool(8) as p:
        vocabul = p.map(read_book, books)
    for i, book in enumerate(books):
        dictionary[book.replace('.txt', '')] = vocabul[i]
        vocabulary += vocabul[i]
        memory += os.stat(dirname + book).st_size
    return memory, vocabulary, dictionary


memory, vocabulary, dictionary = get_vocabulary()
print(f'Total number of words : {len(vocabulary)}')
print(f'Total number of tokens : {len(set(vocabulary))}')

Total number of words : 708449
Total number of tokens : 94292
CPU times: user 662 ms, sys: 384 ms, total: 1.05 s
Wall time: 1.17 s


In [2]:
def check_components(parts, word, state='ordinary'):
    temp = 0
    if state == 'ordinary' and word.find(parts[0]) != 0:
        return False
    if state == 'parts':
        temp = word.find(parts[0])
    for i in range(1, len(parts)):
        ind = word.find(parts[i])
        if temp < ind:
            temp = ind
            continue
        else:
            return False
    return True


def get_intersection(l1, l2):
    return list(set(l1) & set(l2))


def isUkrainian(word):
    ukrainian = [96, 1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093, 1094, 1095, 1096, 1097, 1100, 1102, 1103, 1108, 1110, 1111, 1169, 8217]
    for l in word:
        if ord(l) not in ukrainian:
            return False
    return True
       

In [3]:
class TreeNode():
    def __init__(self):
        self.children = {}
        self.last = False
 
 
class Tree():
    def __init__(self):
        self.root = TreeNode()
 
    def insert(self, key):
        node = self.root
        for a in key:
            if not node.children.get(a):
                node.children[a] = TreeNode()
            node = node.children[a]
        node.last = True
 
    def suggestions_recursive(self, node, word, results):
        if node.last:
            results.append(word)
        for a, n in node.children.items():
            self.suggestions_recursive(n, word + a, results)
 
    def find_suggestions(self, key):
        results = list()
        node = self.root
        for a in key:
            if not node.children.get(a):
                return 0, results
            node = node.children[a]
        if not node.children:
            return -1, results
        self.suggestions_recursive(node, key, results)
        return 1, results




In [4]:

class TreeSearchTool():
    def __init__(self, vocabulary):
        self.tree = Tree()
        self.inverse = Tree()
        self.vocabulary = set(vocabulary)
        self.only_ukrainian = list()
        self.only_ukrainian = self.check_vocabulary()
        self.create_tree()
        self.create_inverse_tree()


    def check_vocabulary(self):
        self.only_ukrainian = [k for k in self.vocabulary if isUkrainian(k) == True]


    def create_inverse_tree(self):
        for word in self.vocabulary:
            self.inverse.insert(word[::-1])
            

    def create_tree(self):
        for word in self.vocabulary:
            self.tree.insert(word)
   

    def search_with_one_jocker(self, string, star='last'):
        index_jocker = string.find("*")
        if index_jocker == -1:
            success, suggestions = self.tree.find_suggestions(string)
            return success , [s for s in suggestions if s == string]
        if index_jocker == len(string) - 1:
            return self.tree.find_suggestions(string[:index_jocker])    
        _, versionsStart = self.tree.find_suggestions(string[:index_jocker])
        end = string[string.rfind("*") + 1:]
        _, versionsEnd = self.inverse.find_suggestions(end[::-1])
        versionsEnd = [i[::-1] for i in versionsEnd]
        inter = get_intersection(versionsStart, versionsEnd)
        if star == 'last':
            return 1, self.check_possible_variants(string.split('*'), inter)
        return 1 , self.check_possible_variants(string.split('*'), inter, star='first')

    
    def check_possible_variants(self, parts, possible_res, star='last', state='ordinary'):
        variants = list()
        for word in possible_res:
            if check_components(parts, word, state) == True:
                variants.append(word)
            else:
                continue
        if star == 'first':
            return [i[::-1] for i in variants]
        return variants
    

    def check_jockers(self, string):
        stars = string.count('*')
        parts = [i for i in string.split('*') if i != '']
        if string[0] != '*' and string[-1] == '*' and len(parts) <= stars:
            _, suggestions = self.tree.find_suggestions(string[:string.find('*')])
            return self.check_possible_variants(parts, suggestions)
        elif string[0] == '*' and len(parts) <= stars and string[-1] != '*':
            shorted = string[1:]
            shorted = shorted[::-1]
            _, suggestions = self.inverse.find_suggestions(shorted[:shorted.find('*')])
            return self.check_possible_variants(shorted.split('*'), suggestions, star='first')
        elif string[0] != '*'  and string[-1] != '*' and stars < 2:
            return self.search_with_one_jocker(string, star='last')
        elif string[0] == '*'  and string[-1] == '*':
            suggestions = [i for i in self.vocabulary if i.find(parts[0]) != -1]
            return self.check_possible_variants(parts, suggestions, state='parts')
        else:
            _, suggestions = self.search_with_one_jocker(string)
            return suggestions

## Define Tree

In [5]:
%%time
tree_search_tool = TreeSearchTool(vocabulary)

CPU times: user 4.35 s, sys: 112 ms, total: 4.46 s
Wall time: 4.46 s


## Check search with different amount and positions of jockers

In [6]:
%%time
found_results = tree_search_tool.check_jockers('*ті*ні*')
print(found_results)

['їстівні', 'тісніше', 'тіснім', 'тіні', 'тіснішим', 'натільній', 'бастіоні', 'тісній', 'тіснині', 'цвітінні', 'втішні', 'отінені', 'стіні', 'ватікані', 'затісні', 'постійні', 'постійність', 'невтішні', 'сутіні', 'тісні', 'тремтінні', 'тітчиній', 'палахтінні', 'затіненій', 'постійній', 'непотішнії']
CPU times: user 35.3 ms, sys: 0 ns, total: 35.3 ms
Wall time: 34.8 ms


In [7]:
%%time
found_results = tree_search_tool.check_jockers('ті*ні*')
print(found_results)

['тітчиній', 'тісні', 'тісніше', 'тіснішим', 'тіснім', 'тісній', 'тіснині', 'тіні']
CPU times: user 450 µs, sys: 0 ns, total: 450 µs
Wall time: 454 µs


In [8]:
%%time
found_results = tree_search_tool.check_jockers('ті*ні')
print(found_results)

(1, ['тіснині', 'тісні', 'тіні'])
CPU times: user 5.67 ms, sys: 0 ns, total: 5.67 ms
Wall time: 5.64 ms


In [9]:
%%time
found_results = tree_search_tool.check_jockers('ті*')
print(found_results)

['ті', 'тібр', 'тібрі', 'тібра', 'тібру', 'тібету', 'тібо', 'тітко', 'тіткою', 'тітки', 'тітку', 'тітка', 'тіточка', 'тітонька', 'тітонько', 'тітусю', 'тітці', 'тітчину', 'тітчиної', 'тітчиній', 'тісні', 'тісніше', 'тіснішим', 'тіснім', 'тісній', 'тісно', 'тіснота', 'тісноту', 'тісноти', 'тісноті', 'тісної', 'тісного', 'тісний', 'тісниною', 'тіснині', 'тіснину', 'тіснини', 'тісних', 'тісна', 'тісну', 'тісненьке', 'тістечко', 'тістечком', 'тістечка', 'тістечками', 'тістечок', 'тісто', 'тіста', 'тісті', 'тіпала', 'тіпалась', 'тіпало', 'тіпалось', 'тіпалося', 'тіпався', 'тіпає', 'тіпається', 'тіпаєтьсябється', 'тіпнувся', 'тім', 'тімаха', 'тімя', 'тімям', 'тімї', 'тімто', 'тік', 'тікич', 'тікичем', 'тікать', 'тікати', 'тікаю', 'тікаючи', 'тікають', 'тікай', 'тікає', 'тікаєш', 'тікала', 'тікали', 'тікав', 'тішилися', 'тішились', 'тішила', 'тішилась', 'тішилася', 'тішило', 'тішити', 'тішитись', 'тішитися', 'тішить', 'тішив', 'тішився', 'тішать', 'тішаться', 'тішачись', 'тіштеся', 'тіл', 'ті

In [11]:
%%time
found_results = tree_search_tool.check_jockers('*ні*')
print(found_results[:10])

['їстівні', 'консервовані', 'механічного', 'ніжинські', 'наталчиній', 'пильні', 'салонівагоні', 'бабиній', 'зачервонілась', 'заґратовані']
CPU times: user 8.95 ms, sys: 1.09 s, total: 1.1 s
Wall time: 1.2 s
